In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
!pip install xgboost
!pip install lightgbm
!pip install imblearn
!pip install borutashap
plt.rcParams["figure.figsize"] = (20,8)

1

In [75]:
df1=pd.read_csv('Survival_time_event.csv', index_col=0)
# df2=pd.read_csv('Treatment.csv', index_col=0)
df3=pd.read_csv('Clinical_Variables.csv', index_col=0)
df4=pd.read_csv('Genetic_alterations.csv', index_col=0)
df5=pd.read_csv('newLabel.csv', index_col=0)

In [76]:
'''
Correlating numerical features of Time data

- outlier value drop
'''

print('outlier of time: ')
print(df1.loc[df1['time'] < 0, 'time'], end='\n\n')

df1_outlier = df1.copy()
df1_outlier.loc[df1_outlier['time'] < 0, 'time'] = abs(df1_outlier.loc[df1_outlier['time'] < 0, 'time'])
print(df1_outlier.describe(), end='\n\n')

outlier of time: 
905   -7.945621
Name: time, dtype: float64

              time        event
count  1000.000000  1000.000000
mean     51.876125     0.891000
std      22.122689     0.311795
min       7.070708     0.000000
25%      37.401307     1.000000
50%      47.064712     1.000000
75%      60.966476     1.000000
max     217.078908     1.000000



In [77]:
'''
Correlating numerical features of Clinic data

- outlier value drop
'''

df3_outlier = df3.copy()


# drop outlier
for col in df3_outlier.columns:
    for outlier in range(10,13):
        df3_outlier = df3_outlier.replace(outlier, 9)

# visualize
for col in df3_outlier.columns:
    print('#', col)
    print(df3_outlier[col].value_counts())
    print('-'*20)

# Var1
2    235
3    204
1    171
4    139
5     95
0     57
6     50
7     27
8     13
9      9
Name: Var1, dtype: int64
--------------------
# Var2
3    221
2    218
4    163
1    113
5    109
6     65
0     48
7     30
9     20
8     13
Name: Var2, dtype: int64
--------------------
# Var3
2    260
3    196
1    156
4    130
5     97
0     55
6     55
7     23
8     16
9     12
Name: Var3, dtype: int64
--------------------
# Var4
2    242
3    195
1    150
4    140
5    106
6     67
0     36
7     32
8     16
9     16
Name: Var4, dtype: int64
--------------------
# Var5
2    247
3    223
4    161
5    124
1     76
6     63
7     41
0     28
9     19
8     18
Name: Var5, dtype: int64
--------------------
# Var6
2    240
3    212
4    128
1    127
5     99
6     64
0     53
7     40
8     20
9     17
Name: Var6, dtype: int64
--------------------
# Var7
1    269
2    208
3    144
0    128
4    118
5     62
6     47
7     16
8      6
9      2
Name: Var7, dtype: int64
--------------------

In [78]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score

In [79]:
'''
Dataset of Best Accuracy


'''

df = pd.concat([df5, df1_outlier, df3_outlier ,df4], axis=1)
df = df.drop(['event'], axis=1)

In [85]:
'''
Model list


'''

ensemble_models = [
    ('lrcv', LogisticRegressionCV(max_iter = 5000)),
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier(n_estimators=20)),
    ('knn', KNeighborsClassifier(n_neighbors = 4)),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('gnb',GaussianNB()),
]

models = [LogisticRegression(max_iter = 10000), SVC(), KNeighborsClassifier(n_neighbors = 4), 
          GaussianNB(), Perceptron(),
          SGDClassifier(), 
          DecisionTreeClassifier(), RandomForestClassifier(n_estimators=20),
          VotingClassifier(ensemble_models, voting='soft')]
#LinearSVC(max_iter=1000000)

In [86]:
'''
Select Model of Best Accuracy


'''
from sklearn.model_selection import StratifiedKFold


def training(model_list):
    best_model = []
    for model in model_list:
        model_name = str(model)[:str(model).find('(')]
        print('Model: ', model_name)
        print()
        features = df.drop(['newlabel'], axis=1)
        labels = df['newlabel']
        
        splits = [5, 10, 7]
        
        for s in splits:
            skfold = StratifiedKFold(n_splits=s)
            idx_iter=0
            cv_accuracy=[]
            cv_precision=[]
            cv_recall=[]
            cv_f1score=[]

            for i in range(20):
                features = features.sample(frac=1).reset_index(drop=True)
                labels = labels.sample(frac=1).reset_index(drop=True)

                for train_index, test_index in skfold.split(features,label):
                    np.random.shuffle(train_index)
                    np.random.shuffle(test_index)

                    # split train and test set
                    X_train, X_test = features.iloc[train_index,:], features.iloc[test_index,:]
                    y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

                    # train ans prediction
                    model.fit(X_train, y_train)
                    pred = model.predict(X_test)

                    idx_iter += 1

                    # 
                    accuracy = np.round(accuracy_score(y_test, pred), 4)
                    cv_accuracy.append(accuracy)

                    precision = np.round(precision_score(y_test, pred, average='weighted', zero_division=0), 4)
                    cv_precision.append(precision)

                    recall = np.round(recall_score(y_test, pred, average='weighted', zero_division=0), 4)
                    cv_recall.append(recall)

                    f1score = np.round(f1_score(y_test, pred, average='weighted', zero_division=0), 4)
                    cv_f1score.append(f1score)

                    #train_size = X_train.shape[0]
                    #test_size = X_test.shape[0]

                    #print('\n#{0} 교차 검증 정확도: {1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'.format(idx_iter, accuracy, train_size, test_size))
                    #print('#{0} 검증 세트 인덱스: MIN{1}, MAX{2}'.format(idx_iter, min(test_index), max(test_index)))

                    #print('학습 레이블 데이터 분포: \n', pd.Series(y_train).value_counts())
                    #print('검증 레이블 데이터 분포: \n', pd.Series(y_test).value_counts())

            print('## 교차 검증 총 횟수: ', len(cv_accuracy), '(분할개수:', s, ')')
            #print('\n## 교차 검증별 정확도: ', np.round(cv_accuracy, 4))
            print('## 평균 검증 정확도: ', np.round(np.mean(cv_accuracy), 5))
            print('## 평균 검증 F1 Score: ', np.round(np.mean(cv_f1score), 5))
            print('##')
            #save model name, split num, acc, pre, rec, f1
            best_model.append([model_name, s, 
                               np.round(np.mean(cv_accuracy), 5), 
                               np.round(np.mean(cv_precision), 5),
                               np.round(np.mean(cv_recall), 5),
                               np.round(np.mean(cv_f1score), 5)])
        print()
        print('-'*100)
        print()

In [87]:
training(models)

Model:  LogisticRegression

## 교차 검증 총 횟수:  100 (분할개수: 5 )
## 평균 검증 정확도:  0.42215
## 평균 검증 F1 Score:  0.41318
##
## 교차 검증 총 횟수:  200 (분할개수: 10 )
## 평균 검증 정확도:  0.4174
## 평균 검증 F1 Score:  0.41079
##
## 교차 검증 총 횟수:  140 (분할개수: 7 )
## 평균 검증 정확도:  0.41504
## 평균 검증 F1 Score:  0.40804
##

----------------------------------------------------------------------------------------------------

Model:  SVC

## 교차 검증 총 횟수:  100 (분할개수: 5 )
## 평균 검증 정확도:  0.43175
## 평균 검증 F1 Score:  0.33068
##
## 교차 검증 총 횟수:  200 (분할개수: 10 )
## 평균 검증 정확도:  0.4312
## 평균 검증 F1 Score:  0.35513
##
## 교차 검증 총 횟수:  140 (분할개수: 7 )
## 평균 검증 정확도:  0.43174
## 평균 검증 F1 Score:  0.33343
##

----------------------------------------------------------------------------------------------------

Model:  KNeighborsClassifier

## 교차 검증 총 횟수:  100 (분할개수: 5 )
## 평균 검증 정확도:  0.42965
## 평균 검증 F1 Score:  0.40835
##
## 교차 검증 총 횟수:  200 (분할개수: 10 )
## 평균 검증 정확도:  0.4238
## 평균 검증 F1 Score:  0.40347
##
## 교차 검증 총 횟수:  140 (분할개수: 7 )
## 평균 검증 정확도

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

C:\Users\enssel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\enssel\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

KeyboardInterrupt: 

In [ ]:
np_best_model = np.array(best_model)
max_model = np.argmax(np_best_model, axis=0)
best_model = best_model[max_model,:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

def modeling(model):
    skfold = StratifiedKFold(n_splits=10)
    cv_accuracy=[]
    cv_f1score=[]

    for i in range(20):
        features = features.sample(frac=1).reset_index(drop=True)
        labels = labels.sample(frac=1).reset_index(drop=True)

        for train_index, test_index in skfold.split(features,label):
            np.random.shuffle(train_index)
            np.random.shuffle(test_index)

            # split train and test set
            X_train, X_test = features.iloc[train_index,:], features.iloc[test_index,:]
            y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

            # train ans prediction
            model.fit(X_train, y_train)
            model.predict(X_test)
            
            accuracy = np.round(accuracy_score(y_test, pred), 4)
            cv_accuracy.append(accuracy)

            f1score = np.round(f1_score(y_test, pred, average='weighted', zero_division=0), 4)
            cv_f1score.append(f1score)

    print('## 교차 검증 총 횟수: ', len(cv_accuracy))
    print('## 평균 검증 정확도: ', np.round(np.mean(cv_accuracy), 5))
    print('## 평균 검증 F1 Score: ', np.round(np.mean(cv_f1score), 5))

modeling(best_model)

In [4]:
'''
Feature selection


'''

from BorutaShap import BorutaShap
import shap
from eli5.lightgbm import *
from eli5.sklearn import *
import eli5

In [ ]:
'''
Feature selection Method 1

BorutaShap

'''

Feature_Selector = BorutaShap(model=best_model, importance_measure='shap', classification=True)
Feature_Selector.fit(X=X_train, y=Y_train, n_trials=50, random_state=0)
Feature_Selector.plot(X_size=12, which_features='all', figsize=(48,24))

In [ ]:
'''
Feature selection Method 2

TreeExplainer

'''

shap.initjs()
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)
fig = shap.summary_plot(shap_values, X_test)

In [ ]:
'''
Feature selection Method 3

PermutationImportance

'''

perm = PermutationImportance(best_model, random_state=42).fit(X_test, y_test)
#print(eli5.format_as_text(explain_weights.explain_permutation_importance(perm, feature_names = X_test.columns.values, top=40)))
explain_weights.explain_permutation_importance(perm, feature_names = X_test.columns.values, top=50)